In [10]:
import os, shutil

# 훈련, 테스트 데이터 directory 저장
train_dir = './data_bill/base'
#validation_dir = ''
test_dir = './data_bill/test'

# train directory
train_1000b_dir = './data_bill/base/1000b'
train_1000f_dir = './data_bill/base/1000f'
train_5000b_dir = './data_bill/base/5000b'
train_5000f_dir = './data_bill/base/5000f'
train_10000b_dir = './data_bill/base/10000b'
train_10000f_dir = './data_bill/base/10000f'
train_50000b_dir = './data_bill/base/50000b'
train_50000f_dir = './data_bill/base/50000f'

# validation directory

# test directory
test_1000b_dir =  './data_bill/test/1000b'
test_1000f_dir = './data_bill/test/1000f'
test_5000b_dir = './data_bill/test/5000b'
test_5000f_dir = './data_bill/test/5000f'
test_10000b_dir = './data_bill/test/10000b'
test_10000f_dir = './data_bill/test/10000f'
test_50000b_dir = './data_bill/test/50000b'
test_50000f_dir = './data_bill/test/50000f'

print('훈련용 1000원 앞면 이미지 전체 개수:', len(os.listdir(train_1000f_dir)))
print('훈련용 1000원 뒷면 이미지 전체 개수:', len(os.listdir(train_1000b_dir)))
print('훈련용 5000원 앞면 이미지 전체 개수:', len(os.listdir(train_5000f_dir)))
print('훈련용 5000원 뒷면 이미지 전체 개수:', len(os.listdir(train_5000b_dir)))
print('훈련용 10000원 앞면 이미지 전체 개수:', len(os.listdir(train_10000f_dir)))
print('훈련용 10000원 뒷면 이미지 전체 개수:', len(os.listdir(train_10000b_dir)))
print('훈련용 50000원 앞면 이미지 전체 개수:', len(os.listdir(train_50000f_dir)))
print('훈련용 50000원 뒷면 이미지 전체 개수:', len(os.listdir(train_50000b_dir)), '\n')
print('테스트용 1000원 앞면 이미지 전체 개수:', len(os.listdir(test_1000f_dir)))
print('테스트용 1000원 뒷면 이미지 전체 개수:', len(os.listdir(test_1000b_dir)))
print('테스트용 5000원 앞면 이미지 전체 개수:', len(os.listdir(test_5000f_dir)))
print('테스트용 5000원 뒷면 이미지 전체 개수:', len(os.listdir(test_5000b_dir)))
print('테스트용 10000원 앞면 이미지 전체 개수:', len(os.listdir(test_10000f_dir)))
print('테스트용 10000원 뒷면 이미지 전체 개수:', len(os.listdir(test_10000b_dir)))
print('테스트용 50000원 앞면 이미지 전체 개수:', len(os.listdir(test_50000f_dir)))
print('테스트용 50000원 뒷면 이미지 전체 개수:', len(os.listdir(test_50000b_dir)))

훈련용 1000원 앞면 이미지 전체 개수: 195
훈련용 1000원 뒷면 이미지 전체 개수: 189
훈련용 5000원 앞면 이미지 전체 개수: 189
훈련용 5000원 뒷면 이미지 전체 개수: 204
훈련용 10000원 앞면 이미지 전체 개수: 192
훈련용 10000원 뒷면 이미지 전체 개수: 189
훈련용 50000원 앞면 이미지 전체 개수: 44
훈련용 50000원 뒷면 이미지 전체 개수: 190 

테스트용 1000원 앞면 이미지 전체 개수: 66
테스트용 1000원 뒷면 이미지 전체 개수: 67
테스트용 5000원 앞면 이미지 전체 개수: 66
테스트용 5000원 뒷면 이미지 전체 개수: 66
테스트용 10000원 앞면 이미지 전체 개수: 62
테스트용 10000원 뒷면 이미지 전체 개수: 67
테스트용 50000원 앞면 이미지 전체 개수: 70
테스트용 50000원 뒷면 이미지 전체 개수: 58


In [19]:
import matplotlib.pyplot as plt
import tensorflow.keras
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing import image # 이미지 전처리 유틸리티 모듈
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16

from scipy import ndimage

conv_base = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(150, 150, 3))

model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

conv_base.trainable = False # conv layer 동결하여 기존 학습된 weight사용

# Data augmentation
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 40,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
    fill_mode = 'nearest')

test_datagen = ImageDataGenerator(rescale=1./255)


'''
fnames = sorted([os.path.join(train_1000f_dir) for fnames in os.listdir(train_1000f_dir)])

img_path = './data_bill/base/1000b/1000b_001.JPG'# 증식할 이미지 선택
img = image.load_img(img_path, target_size=(150, 150)) # 이미지 읽고 크기 변경

x = image.img_to_array(img) # (150, 150, 3) 크기의 넘파이 배열로 변환
x = x.reshape((1, ) + x.shape) # (1, 150, 150, 3) 크기로 변환

# 무한으로 랜덤하게 변환된 이미지 배치 생성
i = 0
for batch in datagen.flow(x, batch_size = 1):
    plt.figure(i)
    imgplot = plt.imshow(image.array_to_img(batch[0]))
    i += 1
    if i % 10 == 0:
        break
plt.show()
'''

train_generator = train_datagen.flow_from_directory(
        train_dir, # 타깃 디렉터리
        target_size=(150, 150), # 모든 이미지의 크기를 150 × 150로 변경
        batch_size=20, # binary_crossentropy 손실을 사용하므로 이진 레이블이 필요하다
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=2e-5),
              metrics=['acc'])

history = model.fit_generator(
    train_generator,
    steps_per_epoch = 100,
    epochs = 100,
    validation_data = validation_generator,
    validation_steps = 50)

Found 1485 images belonging to 9 classes.
Found 561 images belonging to 9 classes.
Epoch 1/100
100/100 [==============================] - 564s 6s/step - loss: -27.7513 - acc: 0.1275 - val_loss: -45.0043 - val_acc: 0.1101
Epoch 2/100
 79/100 [======================>.......] - ETA: 1:18 - loss: -40.4677 - acc: 0.1310

KeyboardInterrupt: 